In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
# pylint: disable=missing-docstring
import sys
sys.path.append('../../')
import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test
import math
import numpy as np
from i3d import InceptionI3d
from utils import *

# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1, 'Batch size.')
flags.DEFINE_integer('num_frame_per_clib', 64, 'Nummber of frames per clib')
flags.DEFINE_integer('crop_size', 224, 'Crop_size')
flags.DEFINE_integer('rgb_channels', 3, 'Channels for input')
flags.DEFINE_integer('classics', 51, 'The num of class')
FLAGS = flags.FLAGS

os.environ["CUDA_VISIBLE_DEVICES"]="0"
outfile = './features_trainval/hmdb_rgb_64/'
if not os.path.exists(outfile):
    os.makedirs(outfile)


def run_training():
    # Get the sets of images and labels for training, validation, and
    # Tell TensorFlow that the model will be built into the default Graph.
    pre_model_save_dir = "./best_models/rgb_30000_trainval_split1"
    if not os.path.exists(pre_model_save_dir):
        print('WARNING !!!!! check the path model')
        exit()
    else:
        test_list_file = '/media/ee401_2/SPML4/temporal_action/I3D-Tensorflow/list/hmdb_list/train_val3.list'
        file = list(open(test_list_file, 'r'))
        num_test_videos = len(file)
        print("Number of test videos={}".format(num_test_videos))
        with tf.Graph().as_default():
            rgb_images_placeholder, _, labels_placeholder, is_training = placeholder_inputs(
                            FLAGS.batch_size * gpu_num,
                            FLAGS.num_frame_per_clib,
                            FLAGS.crop_size,
                            FLAGS.rgb_channels
                            )

            with tf.variable_scope('RGB'):
                logit, _ = InceptionI3d(
                                    num_classes=FLAGS.classics,
                                    spatial_squeeze=True,
                                    final_endpoint='Mixed_5c',
                                    name='inception_i3d'
                                    )(rgb_images_placeholder, is_training)
            norm_score = tf.nn.softmax(logit)

            # Create a saver for writing training checkpoints.
            saver = tf.train.Saver()
            init = tf.global_variables_initializer()

            # Create a session for running Ops on the Graph.
            sess = tf.Session(
                            config=tf.ConfigProto(allow_soft_placement=True)
                            )
            sess.run(init)

        ckpt = tf.train.get_checkpoint_state(pre_model_save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            print ("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            print ("load complete!")

        all_steps = num_test_videos
        top1_list = []
        count = 0
        for step in xrange(all_steps):
            start_time = time.time()
            s_index = 0
            predicts = []
            data = {}
            top1 = False
            data_new = np.array([])
            while True:
                val_images, _, val_labels, s_index, is_end = input_test.read_clip_and_label(
                                filename=file[step],
                                batch_size=FLAGS.batch_size * gpu_num,
                                s_index=s_index,
                                num_frames_per_clip=FLAGS.num_frame_per_clib,
                                crop_size=FLAGS.crop_size,
                                )

                data_features = sess.run(logit,
                                   feed_dict={
                                                rgb_images_placeholder: val_images,
                                                labels_placeholder: val_labels,
                                                is_training: False
                                                })

                #print("feat ================")
                #print(data_features.shape)
                #print("feat ================")

                temp = np.mean(data_features, axis=2)
                tmp = np.mean(temp, axis=2)
                tmp = np.squeeze(tmp)

                #print("feat ================")
                #tmp = data_features
                #print(tmp.shape)
                #print("feat ================")


                if data_new.shape[0] == 0:
                    data_new = tmp
                else:
                    data_new = np.concatenate([data_new,tmp])

                labell = np.zeros((1,51))
                labell[0,int(val_labels)] = 1

                #print(labell)

                if is_end:
                    ndata = int(data_new.shape[0])
                    nn = 64
                    if ndata <=nn:
                        data_new = np.concatenate([data_new,data_new],  axis=0)
                        data_new = np.concatenate([data_new,data_new],  axis=0)
                        ndata = int(data_new.shape[0])

                    sampling = ndata/nn
                    sampling = int(sampling)
                    #print(sampling)
                    #print(data_new.shape)
                    ndata2 = data_new[0:nn*sampling,:]
                    ndata3 = ndata2[0::sampling,:]

                    data['feat'] = ndata3
                    data['label'] = labell

                    temp = file[step].split("/")
                    namefile = temp[6].split(" ")
#                     print(namefile[0])

                    datasave = outfile+namefile[0]
                    print(datasave)
                    np.save(datasave, data)
                    break
            count = count+1
                    
            print(count,' >>' ,namefile[0])


def main(_):
    run_training()


if __name__ == '__main__':
    tf.app.run()


Number of test videos=3029
net_shape:
(1, 8, 7, 7, 1024)
loading checkpoint ./best_models/rgb_30000_trainval_split1/i3d_hmdb_model-29999,waiting......
INFO:tensorflow:Restoring parameters from ./best_models/rgb_30000_trainval_split1/i3d_hmdb_model-29999
load complete!
./features_trainval/hmdb_rgb_64/April_09_brush_hair_u_nm_np1_ba_goo_0
1  >> April_09_brush_hair_u_nm_np1_ba_goo_0
./features_trainval/hmdb_rgb_64/April_09_brush_hair_u_nm_np1_ba_goo_1
2  >> April_09_brush_hair_u_nm_np1_ba_goo_1
./features_trainval/hmdb_rgb_64/April_09_brush_hair_u_nm_np1_ba_goo_2
3  >> April_09_brush_hair_u_nm_np1_ba_goo_2
./features_trainval/hmdb_rgb_64/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4
4  >> Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4
./features_trainval/hmdb_rgb_64/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ri_med_3
5  >> Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ri_med_3
./features_trainval/hmdb_rgb_64/Aussie_Brunette_Brushing_Hair_II_

./features_trainval/hmdb_rgb_64/Trannydude___Brushing_SyntheticHair___OhNOES__those_fukin_knots_brush_hair_u_nm_np1_fr_goo_1
51  >> Trannydude___Brushing_SyntheticHair___OhNOES__those_fukin_knots_brush_hair_u_nm_np1_fr_goo_1
./features_trainval/hmdb_rgb_64/Trannydude___Brushing_SyntheticHair___OhNOES__those_fukin_knots_brush_hair_u_nm_np1_fr_goo_2
52  >> Trannydude___Brushing_SyntheticHair___OhNOES__those_fukin_knots_brush_hair_u_nm_np1_fr_goo_2
./features_trainval/hmdb_rgb_64/atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_0
53  >> atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_0
./features_trainval/hmdb_rgb_64/atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_1
54  >> atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_1
./features_trainval/hmdb_rgb_64/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_0
55  >> brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_0
./features_trainval/hmdb_rgb_64/brush_my_hair_without_wearing_the_g

./features_trainval/hmdb_rgb_64/Ein_guter_Radschlag_cartwheel_f_nm_np1_ba_med_0
108  >> Ein_guter_Radschlag_cartwheel_f_nm_np1_ba_med_0
./features_trainval/hmdb_rgb_64/Ein_guter_Radschlag_cartwheel_f_nm_np1_ba_med_3
109  >> Ein_guter_Radschlag_cartwheel_f_nm_np1_ba_med_3
./features_trainval/hmdb_rgb_64/Ein_guter_Radschlag_cartwheel_f_nm_np1_le_med_1
110  >> Ein_guter_Radschlag_cartwheel_f_nm_np1_le_med_1
./features_trainval/hmdb_rgb_64/Ein_guter_Radschlag_cartwheel_f_nm_np1_le_med_2
111  >> Ein_guter_Radschlag_cartwheel_f_nm_np1_le_med_2
./features_trainval/hmdb_rgb_64/Ein_guter_Radschlag_cartwheel_f_nm_np1_le_med_4
112  >> Ein_guter_Radschlag_cartwheel_f_nm_np1_le_med_4
./features_trainval/hmdb_rgb_64/Estrela_mini_kelly_cartwheel_f_cm_np1_ri_med_0
113  >> Estrela_mini_kelly_cartwheel_f_cm_np1_ri_med_0
./features_trainval/hmdb_rgb_64/Geena_und_die_Radschl_ge___;-_cartwheel_f_cm_np1_fr_med_2
114  >> Geena_und_die_Radschl_ge___;-_cartwheel_f_cm_np1_fr_med_2
./features_trainval/hmdb_rgb_6

./features_trainval/hmdb_rgb_64/Faith_Rewarded_catch_f_cm_np1_fr_med_10
163  >> Faith_Rewarded_catch_f_cm_np1_fr_med_10
./features_trainval/hmdb_rgb_64/Finding_Forrester_3_catch_f_nm_np2_ba_med_7
164  >> Finding_Forrester_3_catch_f_nm_np2_ba_med_7
./features_trainval/hmdb_rgb_64/Finding_Forrester_3_catch_u_cm_np1_fr_goo_11
165  >> Finding_Forrester_3_catch_u_cm_np1_fr_goo_11
./features_trainval/hmdb_rgb_64/Frisbee_catch_f_cm_np1_fr_bad_2
166  >> Frisbee_catch_f_cm_np1_fr_bad_2
./features_trainval/hmdb_rgb_64/Frisbee_catch_f_cm_np1_ri_med_0
167  >> Frisbee_catch_f_cm_np1_ri_med_0
./features_trainval/hmdb_rgb_64/Frisbee_catch_f_cm_np1_ri_med_1
168  >> Frisbee_catch_f_cm_np1_ri_med_1
./features_trainval/hmdb_rgb_64/Goal_Keeping_Tips_catch_f_cm_np1_fr_med_0
169  >> Goal_Keeping_Tips_catch_f_cm_np1_fr_med_0
./features_trainval/hmdb_rgb_64/Goal_Keeping_Tips_catch_f_cm_np1_fr_med_2
170  >> Goal_Keeping_Tips_catch_f_cm_np1_fr_med_2
./features_trainval/hmdb_rgb_64/Goal_Keeping_Tips_catch_u_cm_n

./features_trainval/hmdb_rgb_64/torwart_de-Camp_2009-_Training_hoher_B_lle_mit_Airbodies_catch_f_cm_np1_fr_bad_2
218  >> torwart_de-Camp_2009-_Training_hoher_B_lle_mit_Airbodies_catch_f_cm_np1_fr_bad_2
./features_trainval/hmdb_rgb_64/torwart_de-Camp_2009-_Training_hoher_B_lle_mit_Airbodies_catch_f_cm_np1_fr_bad_3
219  >> torwart_de-Camp_2009-_Training_hoher_B_lle_mit_Airbodies_catch_f_cm_np1_fr_bad_3
./features_trainval/hmdb_rgb_64/torwart_de-Camp_2009_-_Hohe_B_lle_catch_f_cm_np1_ba_med_0
220  >> torwart_de-Camp_2009_-_Hohe_B_lle_catch_f_cm_np1_ba_med_0
./features_trainval/hmdb_rgb_64/torwart_de-Camp_2009_-_Hohe_B_lle_catch_f_cm_np1_ba_med_1
221  >> torwart_de-Camp_2009_-_Hohe_B_lle_catch_f_cm_np1_ba_med_1
./features_trainval/hmdb_rgb_64/torwart_de-Camp_2009_-_Hohe_B_lle_catch_f_cm_np1_ba_med_2
222  >> torwart_de-Camp_2009_-_Hohe_B_lle_catch_f_cm_np1_ba_med_2
./features_trainval/hmdb_rgb_64/torwart_de_-_Torwarttraining_Werder_Bremen_catch_f_cm_np1_ba_bad_0
223  >> torwart_de_-_Torwartt

...

./features_trainval/hmdb_rgb_64/TrumanShow_walk_f_nm_np1_ri_med_6
2877  >> TrumanShow_walk_f_nm_np1_ri_med_6
./features_trainval/hmdb_rgb_64/TrumanShow_walk_f_nm_np1_ri_med_9
2878  >> TrumanShow_walk_f_nm_np1_ri_med_9
./features_trainval/hmdb_rgb_64/TrumanShow_walk_u_cm_np1_ri_med_18
2879  >> TrumanShow_walk_u_cm_np1_ri_med_18
./features_trainval/hmdb_rgb_64/Two_Towers_2_walk_f_cm_np2_le_med_1
2880  >> Two_Towers_2_walk_f_cm_np2_le_med_1
./features_trainval/hmdb_rgb_64/Two_Towers_5_walk_f_nm_np1_fr_bad_1
2881  >> Two_Towers_5_walk_f_nm_np1_fr_bad_1
./features_trainval/hmdb_rgb_64/Two_Towers_6_walk_u_cm_np1_fr_med_0
2882  >> Two_Towers_6_walk_u_cm_np1_fr_med_0
./features_trainval/hmdb_rgb_64/Two_Towers_7_walk_f_cm_np1_fr_med_2
2883  >> Two_Towers_7_walk_f_cm_np1_fr_med_2
./features_trainval/hmdb_rgb_64/UNREPEATABLE_walk_f_nm_np1_le_med_0
2884  >> UNREPEATABLE_walk_f_nm_np1_le_med_0
./features_trainval/hmdb_rgb_64/Veoh_Alpha_Dog_1_walk_f_cm_np1_ba_med_7
2885  >> Veoh_Alpha_Dog_1_walk_f_c

./features_trainval/hmdb_rgb_64/prelinger_they_grow_up_so_fast_1_walk_f_cm_np10_ri_med_2
2946  >> prelinger_they_grow_up_so_fast_1_walk_f_cm_np10_ri_med_2
./features_trainval/hmdb_rgb_64/prelinger_they_grow_up_so_fast_1_walk_f_cm_np1_le_med_3
2947  >> prelinger_they_grow_up_so_fast_1_walk_f_cm_np1_le_med_3
./features_trainval/hmdb_rgb_64/prelinger_they_grow_up_so_fast_1_walk_f_nm_np1_fr_bad_11
2948  >> prelinger_they_grow_up_so_fast_1_walk_f_nm_np1_fr_bad_11
./features_trainval/hmdb_rgb_64/prelinger_they_grow_up_so_fast_1_walk_f_nm_np1_fr_med_5
2949  >> prelinger_they_grow_up_so_fast_1_walk_f_nm_np1_fr_med_5
./features_trainval/hmdb_rgb_64/preparationoffoods256kb_walk_f_cm_np1_ri_med_1
2950  >> preparationoffoods256kb_walk_f_cm_np1_ri_med_1
./features_trainval/hmdb_rgb_64/prideandprejudice1_walk_f_nm_np1_ba_med_0
2951  >> prideandprejudice1_walk_f_nm_np1_ba_med_0
./features_trainval/hmdb_rgb_64/prideandprejudice1_walk_f_nm_np1_fr_med_2
2952  >> prideandprejudice1_walk_f_nm_np1_fr_med_2

./features_trainval/hmdb_rgb_64/Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_1
3011  >> Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_1
./features_trainval/hmdb_rgb_64/Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_2
3012  >> Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_2
./features_trainval/hmdb_rgb_64/Superbad_wave_f_cm_np1_fr_med_4
3013  >> Superbad_wave_f_cm_np1_fr_med_4
./features_trainval/hmdb_rgb_64/UNREPEATABLE_wave_f_nm_np1_fr_med_5
3014  >> UNREPEATABLE_wave_f_nm_np1_fr_med_5
./features_trainval/hmdb_rgb_64/We_Wave_-_Premiere_wave_f_cm_np2_fr_med_0
3015  >> We_Wave_-_Premiere_wave_f_cm_np2_fr_med_0
./features_trainval/hmdb_rgb_64/Wie_man_winkt_wave_u_cm_np1_fr_med_0
3016  >> Wie_man_winkt_wave_u_cm_np1_fr_med_0
./features_trainval/hmdb_rgb_64/_I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_Male_Choir_wave_u_cm_np1_fr_med_0
3017  >> _I_ll_Just_Wave_My_Hand__sung_by_Cornerstone_Male_Choir_wave_u_cm_np1_fr_med_0
.

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
